In [1]:
import time
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, OptimizersConfigDiff, SearchParams


**Passo 1: Carga de Dados (Stress)**

In [2]:
# Conectar ao Qdrant Local (Docker)
client = QdrantClient("localhost", port=6333)
collection_name = "stress_test"

In [3]:
# 1. Recriar Coleção com HNSW Padrão
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

C:\Users\Max\AppData\Local\Temp\ipykernel_66620\3834630099.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [4]:
# 2. Inserir 50.000 vetores aleatórios (Simulando 768 dimensões)
print("Gerando e inserindo 50k vetores (isso pode levar 1 min)…")
vectors = np.random.rand(50000, 768).tolist()
payloads = [{"id": i, "in_stock": i % 2 == 0} for i in range(50000)] # 50% em estoque

Gerando e inserindo 50k vetores (isso pode levar 1 min)…


In [5]:
# Upload em batch
client.upload_collection(
    collection_name=collection_name,
    vectors=vectors,
    payload=payloads,
    batch_size=256
)
print("Carga concluída!")

Carga concluída!


**Passo 2: Benchmark e Tuning de Query**

In [6]:
# Vetor de consulta aleatório
query_vector = np.random.rand(768).tolist()

def benchmark_search(ef_param):
    start = time.time()
    client.query_points(
        collection_name=collection_name,
        query=query_vector,
        search_params=SearchParams(hnsw_ef=ef_param), # O Parâmetro Mágico
        limit=10
    )
    end = time.time()
    return (end - start) * 1000 # ms


In [7]:

# Testar diferentes valores de EF (Esforço de Busca)
print(f"\n{'HNSW EF':<10} | {'Latência (ms)':<15} | {'Comentário'}")
print("-" * 45)

for ef in [16, 64, 128, 256, 512]:
    latencia = benchmark_search(ef)
    comentario = "Rápido (Baixa Precisão)" if ef < 64 else "Lento (Alta Precisão)"
    print(f"{ef:<10} | {latencia:.2f} ms        | {comentario}")


HNSW EF    | Latência (ms)   | Comentário
---------------------------------------------
16         | 74.80 ms        | Rápido (Baixa Precisão)
64         | 56.79 ms        | Lento (Alta Precisão)
128        | 33.74 ms        | Lento (Alta Precisão)
256        | 66.22 ms        | Lento (Alta Precisão)
512        | 46.79 ms        | Lento (Alta Precisão)
